In [131]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
from time import time, sleep
from datetime import datetime
import random
from json import loads
import json
import re

In [81]:
link = 'https://www.eobuwie.com.pl/oferta'

In [82]:
page = requests.get(link)

In [83]:
soup = BeautifulSoup(page.content)

In [84]:
link_article = soup.find(attrs =  {"class": "products-list__link"})['href'][2:]


In [97]:
link_article 

'www.eobuwie.com.pl/buty-adidas-superstar-j-fu7712-ftwwht-cblack-ftwwht.html'

In [212]:
# testowo dowolny inny artykul

link_article = "www.eobuwie.com.pl/buty-reebok-zig-kinetica-ii-h01276-ftwwht-purgry-glablu.html?snrai_campaign=acQIbt76sJLK&snrai_id=f3da7324-41e8-4b71-a3a4-2f0b7b70ff29"

In [213]:
page_article = requests.get("http://" + link_article)

In [214]:
soup_article = BeautifulSoup(page_article.content)


In [215]:
RRP = soup_article.find(attrs = {"class" : "e-product-price__group"}).find(attrs = {"class" : "e-product-price__regular"}).text.strip().replace('\xa0zł',"")

In [216]:
PRICE = soup_article.find(attrs = {"class" : "e-product-price__group"}).find(attrs = {"class" : "e-product-price__special"}).text.strip().replace('\xa0zł',"")

In [217]:
soup_article.find(attrs = {"class" : "e-product-price__group"})

<div class="e-product-price__group">
<div class="e-product-price__regular">
449,00 zł </div>
<div class="e-product-price__special">
349,00 zł </div>
</div>

In [218]:
NAZWA = soup_article.find("meta" ,property="product:mfr_part_no")["content"]

In [219]:
RRP, PRICE, NAZWA

('449,00', '349,00', 'Zig Kinetica II H01276')